# Chapter 2: Systems of Linear Algebraic Equations

*Warning:* This is a critical chapter of the course.

## 2.1 Introduction

### Notation

A system of linear algebraic equations is written as follows:

$$
A_{11}x_1 + A_{12}x_2 + \ldots + A_{1n}x_n = b_1\\
A_{21}x_1 + A_{22}x_2 + \ldots + A_{2n}x_n = b_2\\
.\\
.\\
.\\
A_{n1}x_1 + A_{n2}x_2 + \ldots + A_{nn}x_n = b_n
$$

where $x_i$ are the unknowns.

It can be represented as $\textbf{Ax}=\textbf{b}$, where A is an $n \times n$ matrix, $\textbf{x}$ is a vector of n unknowns and $\textbf{b}$ is a vector of $n$ constants:

$$
    \begin{bmatrix}
        A_{11} & A_{12} & ... & A_{1n} \\
        A_{21} & A_{22} & ... & A_{2n} \\
        ...    & ...    & ... & ...    \\
        A_{n1} & A_{n2} & ... & A_{nn}
    \end{bmatrix}
    \begin{bmatrix}
    x_1\\
    x_2\\
    ...\\
    x_n
    \end{bmatrix}
    =
    \begin{bmatrix}
    b_1\\
    b_2\\
    ...\\
    b_n
    \end{bmatrix}
$$

It is also sometimes written using the $\textit{augmented form}$ combining $\textbf{A}$ and $\textbf{b}$ in a single matrix:
$$
\left[
\begin{array}{cccc|c}
 A_{11} & A_{12} & ... & A_{1n} & b_{1}\\
        A_{21} & A_{22} & ... & A_{2n} & b_{2} \\
        ...    & ...    & ... & ...    & ... \\
        A_{n1} & A_{n2} & ... & A_{nn} & b_{n}
\end{array}
\right]
$$


Let's do a brief [recap on linear algebra](Matrix-algebra.ipynb)...

### Existence and Uniqueness of Solution

A system of $n$ linear algebraic equations has a unique solution if and only if $\textbf{A}$ is $\textit{nonsingular}$, that is: det($\textbf{A}$) $\neq$ 0

#### Exercice 2.1.2 (in class)

* Find examples of 2x2 and 3x3 matrices with a zero determinant.
* Find examples of systems with (1) no solution, (2) an infinite number of solutions.

### Conditioning

When the determinant of $\textbf{A}$ is "very small", small changes in the matrix result in large changes in the solution. In this case, the solution $\underline{\mathrm{cannot\ be\ trusted}}$.

To determine whether the determinant is small, we compare it to a matrix $\textit{norm}$, for instance:

Frobenius norm:

$$
||\textbf{A}||_F = \sqrt{\sum_{i=1}^n\sum_{j=1}^nA_{ij}^2}
$$

Row-sum norm, a.k.a the infinity norm:

$$
||\textbf{A}||_\infty=\max_{1\leq i \leq n}\sum_{j=1}^n|A_{ij}|
$$

The $\textit{condition number}$ of a matrix is defined as:

$$
\mathrm{cond}(\textbf{A}) = ||\textbf{A}||.||\textbf{A}^{-1}||
$$

If the condition number is close to unity, the matrix is well conditioned. On the contrary, a matrix with a large condition number is said to be $\textit{ill-conditioned}$.

Let's write a function to compute the condition number of a matrix:

In [1]:
def norm(a): # infinity norm
    n = 0
    for row in a:
        s = sum(abs(row))
        if s > n:
            n = s
    return n
                
def condition(a):
    from numpy.linalg import inv # this is cheating, we'll see later how to compute the inverse of a matrix
    return norm(a)*norm(inv(a))

And let's compute the condition number of the following matrix:

$$
\textbf{A}=
\begin{bmatrix}
        1 & -1.001 \\
        2.001 & -2  
    \end{bmatrix}
$$

Using our function:

In [3]:
from numpy import array
a = array([[1, -1.001], [2.001, -2]])
condition(a)

4001.000000000308

And using numpy:

In [4]:
from numpy import inf
from numpy.linalg import cond
cond(a, p=inf)

4001.000000000308

In practice, computing $\textbf{A}^{-1}$ is expensive. Conditioning is often gauged by comparing the determinant with the magnitude of the elements in the matrix.

#### Exercice: Effect of ill-conditioning on solutions.

1. Solve the linear system defined by $\textbf{Ax}$=$\textbf{b}$, where:
$
\textbf{A}=
\begin{bmatrix}
        1 & -1.001 \\
        2.001 & -2  
    \end{bmatrix}
$
and
$
\textbf{b}=
\begin{bmatrix}
        3 \\
        7  
    \end{bmatrix}
$


In [7]:
from numpy.linalg import solve # let's cheat for now, we'll program this in the next section

a = array([[1, -1.001], [2.001, -2]])
b = array([3, 7])
solve(a, b)

array([335.55481506, 332.22259247])

2. Solve the linear system defined by $\textbf{Ax}$=$\textbf{b}$, where:
$
\textbf{A}=
\begin{bmatrix}
        1 & -1.002 \\
        2.002 & -2  
    \end{bmatrix}
$
and
$
\textbf{b}=
\begin{bmatrix}
        3 \\
        7  
    \end{bmatrix}
$

In [8]:
from numpy.linalg import solve # let's cheat for now, we'll program this in the next section

a = array([[1, -1.002], [2.002, -2]])
solve(a, b)

array([168.88740839, 165.5562958 ])

### Methods of Solution

There are two classes of methods to solve linear systems:
* Direct methods
* Iterative methods

Direct methods work by applying the following three operations to rewrite the system in a form that permits resolution:
* Exchanging two equations
* Multiplying an equation by a nonzero constant
* Subtracting an equation from another one

Iterative methods start with an initial solution $\textbf{x}$ and refine it until convergence. Iterative methods are generally used when the matrix is very large and sparse, for instance to solve the [PageRank](https://en.wikipedia.org/wiki/PageRank) equations.


#### Overview of Direct Methods

Direct methods are summarized in the Table below:

| Method        | Initial form    | Final form  |
| ------------- |:-------------:| -----:|
| Gauss Elimination      | $\textbf{Ax}=\textbf{b}$ | $\textbf{Ux}=\textbf{c}$ |
| LU decomposition      | $\textbf{Ax}=\textbf{b}$      |   $\textbf{LUx}=\textbf{b}$ |
| Gauss-Jordan Elimination | $\textbf{Ax}=\textbf{b}$      |    $\textbf{Ix}=\textbf{c}$ |

In this table, $\textbf{U}$ represents an upper triangular matrix, $\textbf{L}$ is a lower triangular matrix, and $\textbf{I}$ is the identity matrix. 

A square matrix is called triangular if it contains only zero elements below (upper triangular) or above (lower triangular) the diagonal. For instance, the following matrix is upper triangular:
$$
\textbf{U}=\begin{bmatrix}
1 & 2 & 3 \\
0 & 4 & 5 \\
0 & 0 & 6
\end{bmatrix}
$$
and the following matrix is lower triangular:
$$
\textbf{L}=\begin{bmatrix}
1 & 0 & 0 \\
2 & 3 & 0 \\
4 & 5 & 6
\end{bmatrix}
$$

Systems of the form $\textbf{Lx}$=$\textbf{c}$ can easily be solved by a procedure called $\textit{forward substitution}$: the first equation has only a single unknown, which is easy to solve; after solving the first equation, the second one has only one unknown remaining, and so on.

#### Exercice

Solve the system of linear equations defined by $\textbf{Lx}$=$\textbf{c}$, where:
$
\textbf{L}=
\begin{bmatrix}
        1 & 0 & 0 \\
        2 & 4 & 0 \\
        3 & 1  & 1
    \end{bmatrix}
$ 
and
$
\textbf{b}=
\begin{bmatrix}
        1\\
        3\\
        2
    \end{bmatrix}
$ 



In [5]:
# You can verify your solution as follows, but the point is to do it manually to understand
# how useful triangular matrices are!
from numpy.linalg import solve

a = array([[1, 0, 0], [2, 4, 0], [3, 1, 1]])
b = array([1, 3, 2])
solve(a, b)

array([ 1.  ,  0.25, -1.25])

Likewise, systems of the form $\textbf{Ux}$=$\textbf{c}$ can easily be solved by $\textit{back substitution}$, solving the last equation first. 

Finally, systems of the form $\textbf{LUx}$=$\textbf{c}$ can quickly be solved by solving first $\textbf{Ly}$=$\textbf{c}$ by forward substitution, and then $\textbf{Ux}$=$\textbf{y}$ by back substitution.

### Exercice (Example 2.2 in textbook)

Solve the equations $\textbf{Ax}$=$\textbf{b}$, where:
$$
\textbf{A}=
\begin{bmatrix}
        8 & -6 & 2 \\
        -4 & 11 & -7 \\
        4 & -7  & 6
    \end{bmatrix}
    \quad
\textbf{b}=
\begin{bmatrix}
        28\\
        -40\\
        33
    \end{bmatrix}
$$ 
knowing that the LU decomposition of $\textbf{A}$ is (you should verify this):
$$
\textbf{A}=\textbf{LU}=
\begin{bmatrix}
        2 & 0 & 0 \\
        -1 & 2 & 0 \\
        1 & -1  & 1
    \end{bmatrix}
    \begin{bmatrix}
        4 & -3 & 1 \\
        0 & 4 & -3 \\
        0 & 0  & 2
    \end{bmatrix}
$$


## 2.2 Gauss Elimination

This method consists of two phases:
1. The elimination phase, to transform the equations in the form $\textbf{Ux}$=$\textbf{c}$,
2. The back substitution phase, to solve the equations.

### Elimination phase

The elimination phase multiplies equations by a constant and subtracts them, which is represented as follows:
$$
Eq.(i) \leftarrow Eq.(i) - \lambda Eq.(j)
$$
Equation $\textit{j}$ is called the $\textit{pivot}$.

For instance, let's consider the following equations:

$$
  3x_1+2x_2-7x_3=4  \quad (a)\\
  2x_1-x_2-4x_3=1   \quad (b)\\
  -x_1-3x_2+x_3=3   \quad (c)
$$

We start the process by choosing Equation (a) as the pivot, and chosing $\lambda$ to eliminate $x_1$ from Equations (b) and (c):
$$
Eq.(b) \leftarrow Eq.(b) - \frac{2}{3}Eq.(a)\\
Eq.(c) \leftarrow Eq.(c) - \left( -\frac{1}{3} \right)Eq.(a)
$$

which gives:
$$
  3x_1+2x_2-7x_3=4  \quad (a)\\
  -\frac{7}{3}x_2+\frac{2}{3}x_3=-\frac{5}{3}   \quad (b)\\
  -\frac{7}{3}x_2-\frac{4}{3}x_3=\frac{13}{3}   \quad (c)
$$

We now reiterate the process by choosing Equation (b) as the pivot to eliminate $x_2$ from Equation (c):
$$
Eq.(c) \leftarrow Eq.(c) - Eq.(b)
$$

which gives:
$$
  3x_1+2x_2-7x_3=4  \quad (a)\\
  -\frac{7}{3}x_2+\frac{2}{3}x_3=-\frac{5}{3}   \quad (b)\\
  -2x_3=\frac{18}{3}   \quad (c)
$$

The elimination phase is complete.

### Back substitution

We can now solve the equations by back substitution:
$$
x_3 = -\frac{9}{3} = -3 \quad (c) \\
x_2 = -\frac{3}{7} \left(-\frac{5}{3} + 2 \right) = - \frac{1}{7}\quad (b) \\
x_1 = \frac{1}{3}\left( 4 + \frac{2}{7} - 21\right) = -\frac{39}{7} \quad (a)
$$

Using the augmented notation, the process is written as follows:
$$
\left[
\begin{array}{ccc|c}
3 & 2 & -7 & 4 \\
2 & -1 & -4 & 1 \\
-1 & -3 & 1 & 3
\end{array}
\right]
$$

$$
\left[
\begin{array}{ccc|c}
3 & 2 & -7 & 4 \\
0 & -7/3 & 2/3 & -5/3 \\
0 & -7/3 & -4/3 & 13/3
\end{array}
\right]
$$

$$
\left[
\begin{array}{ccc|c}
3 & 2 & -7 & 4 \\
0 & -7/3 & 2/3 & -5/3 \\
0 & 0 & -2 & 18/3
\end{array}
\right]
$$

### Notes

* The elimination phase leaves the determinant of the matrix unchanged.
* The determinant of a triangular matrix is the product of its diagonal coefficients.

Thus:

$$
\mathrm{det}\left(
\begin{bmatrix}
3 & 2 & -7 \\
2 & -1 & -4 \\
-1 & -3 & 1
\end{bmatrix}
\right) = 3.-\frac{7}{3}.-2 = 14
$$

In [8]:
from numpy.linalg import det
a = array([[3, 2, -7], [2, -1, -4], [-1, -3, 1]])
det(a)

13.999999999999996

### Algorithm for Gauss Elimination Method

#### Elimination Phase

Let's look at the equations at iteration $k$ of the elimination process.

* The first $k$ rows have already been transformed.
* Row $k$ is the pivot row
* Row $i$ is the row being transformed

The augmented coefficient matrix is:
$$
\left[
\begin{array}{ccccccccc|c}
A_{11} & A_{12} & A_{13} & \ldots & A_{1k} & \ldots & A_{1j} & \ldots & A_{1n} & b_1 \\
0      & A_{22} & A_{23} & \ldots & A_{2k} & \ldots & A_{2j} & \ldots & A_{2n} & b_2 \\
0      & 0      & A_{33} & \ldots & A_{3k} & \ldots & A_{3j} & \ldots & A_{3n} & b_3 \\
\ldots \\
0      & 0      & 0 & \ldots & A_{kk} & \ldots & A_{kj} & \ldots & A_{kn} & b_k \\
\hline
\ldots \\
0      & 0      & 0 & \ldots & A_{ik} & \ldots & A_{ij} & \ldots & A_{in} & b_i \\
\ldots \\
0      & 0      & 0 & \ldots & A_{nk} & \ldots & A_{nj} & \ldots & A_{nn} & b_n
\end{array}
\right]
$$
Note: the coefficients are not the one of the original matrix (why?, exception?).


##### Principle
* Use the first $n-1$ rows successively as pivot row $k$.
* For each pivot row $k$, transform row $i$ ($k+1\leq i \leq n$) as follows: 
$$
A_{ij} \leftarrow A_{ij} - \lambda_{ik} A_{jk}, \forall j \in [k, n]
$$
* With $\lambda_{ik}$ such that after transformation, $A_{ik}=0$: 
$$
\lambda_{ik} = \frac{A_{ik}}{A_{kk}}
$$.

##### Implementation

In [4]:
def gauss_elimination(a, b, verbose=False):
    n = len(a)
    assert(n == len(b))
    for k in range(n-1):
        for i in range(k+1, n):
            assert(a[k,k] != 0) # woops, what happens in this case? we'll talk about it later!
            if (a[i,k] != 0): # no need to do anything when lambda is 0
                lmbda = a[i,k]/a[k,k] # lambda is a reserved keyword in Python
                a[i, k:n] = a[i, k:n] - lmbda*a[k, k:n] # list slice operations
                b[i] = b[i] - lmbda*b[k]
            if verbose:
                print(a, b)

##### Example

In [5]:
a = array([[6.0, -4, 1], [-4, 6, -4], [1, -4, 6]])
b = [-14, 36, 6]
gauss_elimination(a, b)
print(a,b)

[[ 6.         -4.          1.        ]
 [ 0.          3.33333333 -3.33333333]
 [ 0.          0.          2.5       ]] [-14, 26.666666666666668, 35.0]


#### Back substition phase

After the elimination phase, the augmented matrix has the following form:
$$
\left[
\begin{array}{ccccc|c}
A_{11} & A_{12} & A_{13} & \ldots & A_{1n} & b_1 \\
0      & A_{22} & A_{23} & \ldots & A_{2n} & b_2 \\
0      & 0      & A_{33} & \ldots & A_{3n} & b_3 \\
\ldots\\
0      & 0      & 0 & \ldots & A_{nn} & b_n
\end{array}
\right]
$$

The equations are solved from the last row to the first:
* $x_n$ = $b_n/A_{nn}$
* $\forall i \leq n, \quad x_i = \left( b_i - \sum_{j = i +1}^n{A_{ij}x_{j}} \right) \frac{1}{A_{ii}}$

##### Implementation

In [6]:
from numpy import dot, zeros
def gauss_substitution(a, b):
    n = len(a)
    assert(len(b)==n)
    x = zeros(n)
    for k in range(n):
        i = n - 1 - k
        x[i] = (b[i] - dot(a[i,i+1:], x[i+1:]))/a[i,i]
    return x

##### Example

In [8]:
a = array([[6.0, -4, 1], [-4, 6, -4], [1, -4, 6]])
b = [-14, 36, 6]
gauss_elimination(a, b)
gauss_substitution(a, b)

array([10., 22., 14.])

#### Complete Solver

In [11]:
def gauss(a, b):
    gauss_elimination(a, b)
    return gauss_substitution(a, b)

In [12]:
a = array([[6, -4, 1], [-4, 6, -4], [1, -4, 6]])
b = [-14, 36, 6]
gauss(a, b)

array([12.34259259, 26.38888889, 17.5       ])

### Operations count

Counting multiplications and divisions only:

Elimination phase:
$$
\sum_{k=1}^n \sum_{i=k+1}^n (n-k) + 2 = \sum_{k=1}^n (n-k)^2 + 2(n-k) = \sum_{k=0}^{n-1} k^2 + 2\sum_{k=0}^{n-1} k
=\frac{(n-1)n(2n-1)}{6}+n(n-1) = \frac{n(n-1)(2n+5)}{6} = O\left(\frac{n^3}{3}\right)
$$

Backsubstitution phase:
$\sum_{k=1}^n k=\frac{n(n+1)}{2}=O\left(\frac{n^2}{2}\right)$

### Multiple Sets of Equations

It is frequently necessary to solve $\textbf{Ax}=\textbf{b}$ for multiple values of $\textbf{b}$. We denote multiple sets of equations by 
$\textbf{AX}=\textbf{B}$, where:

$$
\textbf{X}=[\textbf{x}_1, \textbf{x}_2, \ldots \textbf{x}_m] \quad \textbf{B}=[\textbf{b}_1, \textbf{b}_2, \ldots \textbf{b}_m]
$$
are $n \times m$ matrices.

An easy way to handle such equations during the elimination phase is to include all constant vectors in the augmented matrix, so that they are transformed simultaneously with the coefficient matrix. $\underline{\mathrm{The\ LU\ decomposition\ method\ provides\ a\ more versatile\ solution}}$.


## 2.3 LU Decomposition Methods

### Introduction

Any square matrix $\textbf{A}$ can be expressed as a product of a lower triangular matrix $\textbf{L}$ and an upper triangular matrix $\textbf{U}$:
$$
\textbf{A}=\textbf{LU}
$$

However, this decomposition is not unique unless certain constraints are placed on $\textbf{L}$ and $\textbf{U}$. Common decomposition methods are listed here:

| Name        | Constraints    |
| ------------- |:-------------:|
| Doolittle's decomposition  | $\textbf{L}_{ii}=1$ |
| Crout's decomposition      | $\textbf{U}_{ii}=1$      |
| Choleski decomposition | $\textbf{L}=\textbf{U}^T$ |

From the decomposition of $\textbf{A}$, it is easy to solve $\textbf{A}\textbf{x}=\textbf{b}$:
1. Solve $\textbf{Ly}=\textbf{b}$ by forward substitution
2. Solve $\textbf{Ux}=\textbf{y}$ by back substitution

The advantage of the $\textbf{LU}$ decomposition method is that once $\textbf{A}$ is decomposed, the cost of solving the system for another instance of $\textbf{b}$ is relatively small.

### Doolittle's Decomposition Method

#### Decomposition phase

This method is closely related to Gauss elimination. Let's consider a $3 \times 3$ matrix $\textbf{A}$ and let's assume that there exists $\textbf{L}$ and $\textbf{U}$:
$$
\textbf{L} = \begin{bmatrix}
1 & 0 & 0 \\
L_{21} & 1 & 0 \\
L_{31} & L_{32} & 1
\end{bmatrix}
\quad
\textbf{U} = \begin{bmatrix}
U_{11} & U_{12} & U_{13}\\
0      & U_{22} & U_{23}\\
0      & 0      & U_{33}
\end{bmatrix}
$$
such that $\textbf{A}=\textbf{LU}$. We get:
$$
\textbf{A} = \begin{bmatrix}
U_{11} & U_{12} & U_{13} \\
L_{21}U_{11} & L_{21}U_{12}+U_{22} & L_{21}U_{13}+U_{23}\\
L_{31}U_{11} & L_{31}U_{12}+L_{32}U_{22} & L_{31}U_{13}+L_{32}U_{23}+U_{33}
\end{bmatrix}
$$

Let's now apply Gauss elimination.

First pass:
* row2 $\leftarrow$ row2 - $L_{21}$row1
* row3 $\leftarrow$ row3 - $L_{31}$row1

$$
\textbf{A} = \begin{bmatrix}
U_{11} & U_{12} & U_{13} \\
0 & U_{22} & U_{23}\\
0 & L_{32}U_{22} & L_{32}U_{23}+U_{33}
\end{bmatrix}$$

Second pass:
* row3 $\leftarrow$ row3-$L_{32}$row2

$$
\textbf{A} = \begin{bmatrix}
U_{11} & U_{12} & U_{13} \\
0 & U_{22} & U_{23}\\
0 & 0 & U_{33}
\end{bmatrix}$$

This illustration shows 2 properties of Doolittle's decomposition:
1. $\textbf{U}$ is the upper diagonal matrix resulting from Gauss decomposition
2. The off-diagonal elements of $\textbf{L}$ are the multipliers for the pivot equations: $L_{ij}$ is used to eliminate $A_{ij}$

It is common to store the multipliers in the lower triangular portion of the matrix. The final form of the coefficient matrix would be:
$$
[\textbf{L} \backslash \textbf{U}]= \begin{bmatrix}
U_{11} & U_{12} & U_{13} \\
L_{21} & U_{22} & U_{23} \\
L_{31} & L_{32} & U_{33}
\end{bmatrix}
$$
The algorithm is very similar to Gauss elimination:

In [21]:
def dolittle_decomp(a, verbose=False):
    n = len(a)
    for k in range(n-1):
        for i in range(k+1, n):
            assert(a[k,k] != 0) # woops, what happens in this case? we'll talk about it later!
            if (a[i,k] != 0): # no need to do anything when lambda is 0
                lmbda = a[i,k]/a[k,k] # lambda is a reserved keyword in Python
                a[i, k:n] = a[i, k:n] - lmbda*a[k, k:n] # list slice operations
                a[i, k] = lmbda # <--- new in Dolittle decomposition
            if verbose:
                print(a)
    return a

In [22]:
from numpy import array
a = array([[1.0, 4, 1], [1, 6, -1], [2, -1, 2]])
dolittle_decomp(a)
print(a)

[[ 1.   4.   1. ]
 [ 1.   2.  -2. ]
 [ 2.  -4.5 -9. ]]


#### Solution phase

First we solve $\textbf{Ly}=\textbf{b}$ by forward substitution:
$$
y_1 = b_1 \\
L_{21}y_1 + L_{22}y_2 = b_2 \\
\ldots
L_{k1}y_1 + L_{k2}y_2 + \ldots + L_{k,k-1}y_{k-1} + y_k = b_k \\
\ldots
$$

Thus:
$$
y_k = b_k - \sum_{i=1}^{k-1}L_{k,i}y_i
$$

Implementation:

In [23]:
def dolittle_solution_forward(l, b):
    for k in range(1, len(b)):
        b[k] = b[k] - dot(l[k,0:k],b[0:k])
    return b

The back substitution phase for solving $\textbf{Ux}=\textbf{y}$ is identical to the one in the Gauss elimination method.

#### Complete solver

In [27]:
def dolittle_solver(a, b, verbose=False):
    dolittle_decomp(a)
    if verbose:
        print("LU={}".format(a))
    y = dolittle_solution_forward(a, b)
    return gauss_substitution(a, y)

#### Exercise

Use Dolittle's decomposition to solve $\textbf{Ax}=\textbf{b}$ where:
$$
\textbf{A} = \begin{bmatrix}
1 & 4 & 1 \\
1 & 6 & -1 \\
2 & -1 & 2
\end{bmatrix}
\quad
\textbf{b} = \begin{bmatrix}
7 \\
13 \\
5
\end{bmatrix}
$$

In [29]:
from numpy import array
a = array([[1.0, 4, 1], [1, 6, -1], [2, -1, 2]])
b = array([7, 13, 5])
dolittle_solver(a, b, True)

LU=[[ 1.   4.   1. ]
 [ 1.   2.  -2. ]
 [ 2.  -4.5 -9. ]]


array([ 5.,  1., -2.])